In [1]:
"""
This notebook plots the predicted track for a super core promoter (Fig. 6A).
"""

'\nThis notebook plots the predicted track for a super core promoter (Fig. 6A).\n'

In [2]:
import json
import numpy as np
import os
import h5py
import sys
sys.path.append("../")
from utils import plot_side

In [3]:
# PRINT COMMANDS TO DOWNLOAD AND EXTRACT DATA

# Set SCRATCH to where you want to download data to
SCRATCH = "/Users/adamhe/github/scratch"

URL = "https://zenodo.org/records/10597358/files"
TAR = "example_tracks_and_deepshap.tar.gz"
print(f"wget {URL}/{TAR} -P {SCRATCH}")
print(f"tar -xvzf {SCRATCH}/{TAR} -C {SCRATCH}")

wget https://zenodo.org/records/10597358/files/example_tracks_and_deepshap.tar.gz -P /Users/adamhe/github/scratch
tar -xvzf /Users/adamhe/github/scratch/example_tracks_and_deepshap.tar.gz -C /Users/adamhe/github/scratch


In [4]:
# Load predictions
prediction = h5py.File(os.path.join(SCRATCH, "example_tracks_and_deepshap/folds_chen_core_promoter_predictions.h5"))
tracks = prediction["track"]
quantity = prediction["quantity"]
quantity = 10 ** (np.hstack(
    [np.log10(quantity[:]), np.ones(quantity.shape[0]).reshape(-1, 1)]
) @ np.array([[1.59687745, -0.80203685]]).T)
y_norm = tracks / np.array(tracks).sum(axis=1, keepdims=True)
y_pred_scaled = y_norm * quantity

In [5]:
# Crop to only core promoter sequences

with open("chen_core_promoter_sequences.json") as f:
    core_promoters = json.load(f)

left_padding = [int(np.floor((1000 - len(seq)) / 2)) for seq in core_promoters.values()]
right_padding = [int(np.ceil((1000 - len(seq)) / 2)) for seq in core_promoters.values()]

In [8]:
plot_side(
    y_pred_scaled[0][
        np.r_[
            left_padding[0] - 250: 750 - right_padding[0],
            500 + left_padding[0] - 250: 500 + 750 - right_padding[0]
        ]
    ],
    ylim=[-1, 4],
    yticks=[0, 4],
    pic_name="img/ensemble_scp_pred.pdf",
)